## Notebook Basics
* Command mode $\leftrightarrow$ Edit mode: `Esc` to command mode, `Enter` to edit mode
* Code $\leftrightarrow$ Markdown: `Esc` + `M` to markdown, `Esc` + `Y` to code, `Enter` to edit
In command mode:
* `H` show help
* Run: 
    * `shift` + `Enter` run current & go to next
    * `ctrl` + `Enter` run current
    * `option` + `Enter` run current & insert below

* Cell:
    * `A`/`B` insert above/below
    * `X`/`C`/`V`/`D+D`, cut/copy/paste/delete
    * `ctrl` + `shift` + `-` split cell (in edit mode)
    * `shift` + `M` merge cells (with below)
    * `shift` + `+`/`-` select cells, or `shift` + click to select
    * `L` line number

## Load Pretrained ResNet101

In [67]:
%load_ext autoreload
%autoreload 2

import torch.nn as nn
from config import Config as cfg
from resnet import resnet_loader

resnet = resnet_loader(arch='resnet101', pretrained=True)

cnn1 = nn.Sequential(
    resnet.conv1,
    resnet.bn1,
    resnet.relu,
    resnet.maxpool,
    resnet.layer1,
    resnet.layer2,
    resnet.layer3
)
# Freeze all pretrained layers
for param in cnn1.parameters():
    param.requires_grad = False
    
cnn2 = nn.Sequential(
    resnet.layer4,
    #nn.AvgPool2d(cfg.POOLING_SIZE) # 7
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
model exists. loading at: resnet101-5d3b4d8f.pth


## Unit Tests for Faster RCNN

In [68]:
%load_ext autoreload
%autoreload 2 # put this line in cell s.t. local changes will be reflected in notebook w/o restart kernel

import torch

N = 2 # use small batch size for test
C, c = 3, 1024
H, W = 600, 800
h, w = round(H/16.0), round(W/16.0)
N_GT = 20 # each image has 20 gt boxes, thus 20 gt class labels as well
N_CLASS = 21 # 21 different classes

images = torch.abs(torch.randn(N*C*H*W).view(N,C,H,W))
feature_map = torch.abs(torch.randn(N*c*h*w).view(N,c,h,w))
gt_boxes = torch.ones(N,N_GT,4).float() # remember to make gt_boxes as float!
gt_boxes[:,:,2] = 50
gt_boxes[:,:,3] = 50
gt_classes = torch.randint(0,N_CLASS, (N,N_GT))

print("images.shape:", images.shape)
print("feature_map.shape:", feature_map.shape)
print("gt_boxes.shape:", gt_boxes.shape)
print("gt_classes.shape:", gt_classes.shape)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
images.shape: torch.Size([2, 3, 600, 800])
feature_map.shape: torch.Size([2, 1024, 38, 50])
gt_boxes.shape: torch.Size([2, 20, 4])
gt_classes.shape: torch.Size([2, 20])


### 1.1 Anchor Generation

In [69]:
from rpn.anchor_generation import AnchorGeneration

anchors = AnchorGeneration().generate_all()
print("anchors.shape:", anchors.shape, "dtype", anchors.type())

anchors.shape: torch.Size([17100, 4]) dtype torch.FloatTensor


### 1.2 Proposal

In [70]:
import torch
from rpn.proposal import Proposal

A = anchors.size(0)

bbox_score = torch.randn(N*A*2).view(N,A,2)
bbox_coeff = torch.randn(N*A*4).view(N,A,4)
print("anchors.shape:", anchors.shape)
print("bbox_score.shape:", bbox_score.shape)
print("bbox_coeff.shape:", bbox_coeff.shape)

anchors.shape: torch.Size([17100, 4])
bbox_score.shape: torch.Size([2, 17100, 2])
bbox_coeff.shape: torch.Size([2, 17100, 4])


In [71]:
rois_scores, rois = Proposal().forward(anchors, bbox_score, bbox_coeff)
print("rois_scores.shape:", rois_scores.shape)
print("rois.shape:", rois.shape)

rois_scores.shape: torch.Size([2, 300, 1])
rois.shape: torch.Size([2, 300, 4])


### 1.3 Anchor Refinement

In [72]:
%autoreload 2
from rpn.anchor_refine import AnchorRefine

labels, target_coeffs, anchors_idx = AnchorRefine().forward(anchors, gt_boxes)
print("labels.shape:", labels.shape)
print("target_coeffs.shape:", target_coeffs.shape)
print("anchors_idx.shape:", anchors_idx.shape)

labels.shape: torch.Size([2, 5740])
target_coeffs.shape: torch.Size([2, 5740, 4])
anchors_idx.shape: torch.Size([5740])


### 1.4 Proposal Refinement

In [73]:
%autoreload 2
from rpn.proposal_refine import ProposalRefine

rois_test, rois_labels, rois_coeffs = ProposalRefine().forward(rois, gt_boxes, gt_classes)
print("rois.shape:", rois_test.shape)
print("rois_labels.shape:", rois_labels.shape)
print("rois_coeffs.shape:", rois_coeffs.shape)

rois.shape: torch.Size([2, 128, 4])
rois_labels.shape: torch.Size([2, 128])
rois_coeffs.shape: torch.Size([2, 128, 84])


### 1. RPN

In [74]:
%autoreload 2

from rpn.rpn import RPN

rois_rpn, rois_labels, rois_coeffs, rpn_loss = RPN().forward(feature_map, gt_boxes, gt_classes)
print("rois_rpn.shape:", rois_rpn.shape)
print("rois_labels.shape:", rois_labels.shape)
print("rois_coeffs.shape:", rois_coeffs.shape)
print("rpn_loss:", rpn_loss)

rois_rpn.shape: torch.Size([2, 128, 4])
rois_labels.shape: torch.Size([2, 128])
rois_coeffs.shape: torch.Size([2, 128, 84])
rpn_loss: tensor(0.9668, grad_fn=<DivBackward0>)


### 2. RoI Pooling

In [75]:
%autoreload 2

from roi_pooling import RoIPooling

crops = RoIPooling().forward(rois_test, feature_map)
print("crops.shape:", crops.shape)

crops.shape: torch.Size([2, 128, 1024, 7, 7])


### 3. Classification

In [76]:
%autoreload 2

from classification import Classification

pred_rois_scores, pred_rois_coeffs = Classification(cnn2).forward(crops)
print("pred_rois_scores.shape:", pred_rois_scores.shape)
print("pred_rois_coeffs.shape:", pred_rois_coeffs.shape)

pred_rois_scores.shape: torch.Size([2, 128, 21])
pred_rois_coeffs.shape: torch.Size([2, 128, 84])


## Faster R-CNN

In [77]:
%autoreload 2

from model import FasterRCNN

rois_rcnn, pred_rois_classes, pred_rois_coeffs, rcnn_loss = FasterRCNN().forward(images, gt_boxes, gt_classes)
print("rois_rcnn.shape:", rois_rcnn.shape)
print("pred_rois_classes.shape:", pred_rois_classes.shape)
print("pred_rois_coeffs.shape:", pred_rois_coeffs.shape)
print("rcnn_loss:", rcnn_loss)

model exists. loading at: resnet101-5d3b4d8f.pth
rois_rcnn.shape: torch.Size([2, 128, 4])
pred_rois_classes.shape: torch.Size([2, 128])
pred_rois_coeffs.shape: torch.Size([2, 128, 84])
rcnn_loss: tensor(4.0090, grad_fn=<AddBackward0>)
